---
# Régression linéaire : `ozone_complet`
---

## Packages

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sys

from sklearn.metrics import mean_absolute_percentage_error, root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LinearRegression

from statsmodels.api import OLS
from statsmodels.tools import add_constant

## Fonctions

In [ ]:
# En local :
directory = '/Users/vincentlefieux/Dropbox/Docs_ACADEMIQUE/Codes/_Python/_Fonctions/'

# Sur Google collab ou Onyxia (sur un répertoire temporaire) :
# directory = ''

# Sur Google collab (sur le drive) :
# from google.colab import drive
# drive.mount('/content/drive')
# directory = '/content/drive/MyDrive/Fonctions/Python/'

In [ ]:
sys.path.append(directory)

import _regression_lineaire as lr

---
## 1. Données
---


### 1.1. Importation

In [ ]:
# En local :
directory = '/Users/vincentlefieux/Dropbox/Docs_ACADEMIQUE/Data/'

# Sur Google collab ou Onyxia (sur un répertoire temporaire) :
# directory = ''

# Sur Google collab (sur le drive) :
# from google.colab import drive
# drive.mount('/content/drive')
# directory = '/content/drive/MyDrive/Data/'

In [ ]:
data = pd.read_csv(directory + 'ozone_complet.csv',
                   header    = 0,
                   index_col = 0,
                   sep       = ';',
                   decimal   = ',')

In [ ]:
data.info()

In [ ]:
data.head()

### 1.2. Gestion des données manquantes

Vu que les données manquantes concernent essentiellement le pic d'ozone qu'on cherche à prédire, on retire les données manquantes ici :

In [ ]:
missing_percentage = data.isna().mean() * 100

print("MISSING VALUES :")
if missing_percentage[missing_percentage != 0].empty:
    print("No")
else:
    print(missing_percentage[missing_percentage != 0].sort_values(ascending=False))

In [ ]:
data.dropna(inplace=True)

### 1.3. Gestion des variables

In [ ]:
target = 'maxO3'

y = data[target]
X = data.drop(target, axis=1)

### 1.4. Création d'un échantillon de test

In [ ]:
test_portion = 1/5

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_portion, shuffle=True)

print('Dimensions X_train :', X_train.shape)
print('Dimensions X_test  :', X_test.shape)
print('Dimensions y_train :', y_train.shape)
print('Dimensions y_test  :', y_test.shape)

---
## 2. Régression linéaire
---

### 2.1. Modèle complet (apprentissage / test)

#### 2.1.1. Estimation

##### 2.1.1.1. Estimation avec `statsmodels`

La constante n'est pas considérée par défaut dans la commande `OLS` de `statsmodels`, il faut l'ajouter.

In [ ]:
linreg_all_model = OLS(y_train, add_constant(X_train))
linreg_all = linreg_all_model.fit()
linreg_all.summary()

##### 2.1.1.2. Estimation avec  `sklearn`

La constante est par contre considérée par défaut dans la commande `LinearRegression` de `sklearn`.

In [ ]:
linreg_all_model_sk = LinearRegression()
linreg_all_sk = linreg_all_model_sk.fit(X_train, y_train)
print('Constante :', linreg_all_sk.intercept_)
print('Coefficients :', linreg_all_sk.coef_)

Si `stasmodels` produit les éléments nécessaires à l'interprétation statistique d'un modèle de régression, il n'en est pas de même pour `sklearn` pour lequel les sorties sont minimalistes.

#### 2.1.2. Qualité prédictive

In [ ]:
y_test_pred_linreg_all = linreg_all.predict(add_constant(X_test))

RMSE_linreg_all = root_mean_squared_error(y_test, y_test_pred_linreg_all)
MAPE_linreg_all = mean_absolute_percentage_error(y_test, y_test_pred_linreg_all) * 100

print(f'RMSE régression linéaire complète : {RMSE_linreg_all:.2f}')
print(f'MAPE régression linéaire complète : {MAPE_linreg_all:.2f}')

### 2.2. Modèle complet (validation croisée)

In [ ]:
n_folds = 10

fold = np.random.randint(low=0, high=n_folds, size=data.shape[0])

nb = np.empty(n_folds, dtype=int)
RMSE_linreg_all_CV = np.empty(n_folds, dtype=float)
MAPE_linreg_all_CV = np.empty(n_folds, dtype=float)

for i in range(n_folds):
  
  nb[i] = X[fold == i].shape[0]

  X_train_CV = X[fold != i]
  y_train_CV = y[fold != i]

  X_test_CV = X[fold == i]
  y_test_CV = y[fold == i]

  linreg_all_CV_model = OLS(y_train_CV, add_constant(X_train_CV))
  linreg_all_CV = linreg_all_CV_model.fit()
  
  y_test_CV_pred = linreg_all_CV.predict(add_constant(X_test_CV))

  RMSE_linreg_all_CV[i] = root_mean_squared_error(y_test_CV, y_test_CV_pred)
  MAPE_linreg_all_CV[i] = mean_absolute_percentage_error(y_test_CV, y_test_CV_pred) * 100

RMSE_linreg_all_CV_g = np.sum(nb * RMSE_linreg_all_CV / np.sum(nb))
MAPE_linreg_all_CV_g = np.sum(nb * MAPE_linreg_all_CV / np.sum(nb))

print(f'RMSE CV régression linéaire complète : {RMSE_linreg_all_CV_g:.2f}')
print(f'MAPE CV régression linéaire complète : {MAPE_linreg_all_CV_g:.2f}')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(range(n_folds), RMSE_linreg_all_CV)
ax.axhline(y=RMSE_linreg_all_CV_g, label='RMSE CV', color='red')
#ax.grid()
ax.set_xticks(range(n_folds))
ax.set_xlabel('Bloc')
ax.set_ylabel('RMSE')
ax.legend(loc='best')
plt.title('Régression linéaire complète (CV)')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(range(n_folds), MAPE_linreg_all_CV)
ax.axhline(y=MAPE_linreg_all_CV_g, label='MAPE CV', color='red')
#ax.grid()
ax.set_xticks(range(n_folds))
ax.set_xlabel('Bloc')
ax.set_ylabel('MAPE')
ax.legend(loc='best')
plt.title('Régression linéaire complète (CV)')
plt.show()

### 2.3. Modèle avec sélection de variables (apprentissage / test)

In [ ]:
linreg_backward = lr.linreg_backward_proc(add_constant(X_train), y_train, crit='BIC', verbose=True)

In [ ]:
linreg_forward = lr.linreg_forward_proc(add_constant(X_train), y_train, crit='BIC', verbose=True)

In [ ]:
linreg_stepwise = lr.linreg_stepwise_proc(add_constant(X_train), y_train, crit='BIC', verbose=True)

In [ ]:
print('Nombre de paramètres du modèle backward :', linreg_backward.params.shape[0])
print('Nombre de paramètres du modèle forward  :', linreg_forward.params.shape[0])
print('Nombre de paramètres du modèle stepwise :', linreg_stepwise.params.shape[0])

In [ ]:
print('Covariables du modèle backward :', linreg_backward.model.exog_names)
print('Covariables du modèle forward  :', linreg_forward.model.exog_names)
print('Covariables du modèle stepwise :', linreg_stepwise.model.exog_names)

In [ ]:
print(f'BIC du modèle backward : {linreg_backward.bic:.2f}')
print(f'BIC du modèle forward  : {linreg_forward.bic:.2f}')
print(f'BIC du modèle stepwise : {linreg_stepwise.bic:.2f}')

In [ ]:
y_test_pred_linreg_backward = linreg_backward.predict(add_constant(X_test)[linreg_backward.model.exog_names])
y_test_pred_linreg_forward  = linreg_forward.predict(add_constant(X_test)[linreg_forward.model.exog_names])
y_test_pred_linreg_stepwise = linreg_stepwise.predict(add_constant(X_test)[linreg_stepwise.model.exog_names])

RMSE_linreg_backward = root_mean_squared_error(y_test, y_test_pred_linreg_backward)
RMSE_linreg_forward  = root_mean_squared_error(y_test, y_test_pred_linreg_forward)
RMSE_linreg_stepwise = root_mean_squared_error(y_test, y_test_pred_linreg_stepwise)

print(f'RMSE régression linéaire backward : {RMSE_linreg_backward:.2f}')
print(f'RMSE régression linéaire forward  : {RMSE_linreg_forward:.2f}')
print(f'RMSE régression linéaire stepwise : {RMSE_linreg_stepwise:.2f}')

In [ ]:
MAPE_linreg_backward = mean_absolute_percentage_error(y_test, y_test_pred_linreg_backward) * 100
MAPE_linreg_forward  = mean_absolute_percentage_error(y_test, y_test_pred_linreg_forward) * 100
MAPE_linreg_stepwise = mean_absolute_percentage_error(y_test, y_test_pred_linreg_stepwise) * 100

print(f'MAPE régression linéaire backward : {MAPE_linreg_backward:.2f}')
print(f'MAPE régression linéaire forward  : {MAPE_linreg_forward:.2f}')
print(f'MAPE régression linéaire stepwise : {MAPE_linreg_stepwise:.2f}')

### 2.4. Modèle avec *feature engineering* (apprentissage / test)

On ajoute les termes polynomiaux de degré 2 et les interactions.

#### 2.4.1. Sans sélection de variable 

In [ ]:
transform_poly_interac = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

X_train_fe_array = transform_poly_interac.fit_transform(X_train)
X_train_fe_ind = X_train.index
X_train_fe_col = transform_poly_interac.get_feature_names_out(X_train.columns)
X_train_fe = pd.DataFrame(X_train_fe_array, index=X_train_fe_ind, columns=X_train_fe_col)

X_test_fe_array = transform_poly_interac.fit_transform(X_test)
X_test_fe_ind = X_test.index
X_test_fe_col = transform_poly_interac.get_feature_names_out(X_test.columns)
X_test_fe = pd.DataFrame(X_test_fe_array, index=X_test_fe_ind, columns=X_test_fe_col)

In [ ]:
linreg_all_model_fe = OLS(y_train, add_constant(X_train_fe))
linreg_all_fe = linreg_all_model_fe.fit()
linreg_all_fe.summary()

Le coefficient de détermination de ce modèle est plus élevé, il faut néanmoins étudier sa capacité à généraliser. 

In [ ]:
y_test_pred_linreg_all_fe = linreg_all_fe.predict(add_constant(X_test_fe))

RMSE_linreg_all_fe = root_mean_squared_error(y_test, y_test_pred_linreg_all_fe)
MAPE_linreg_all_fe = mean_absolute_percentage_error(y_test, y_test_pred_linreg_all_fe) * 100

print(f'RMSE régression linéaire complète FE : {RMSE_linreg_all_fe:.2f}')
print(f'MAPE régression linéaire complète FE : {MAPE_linreg_all_fe:.2f}')

#### 2.4.2. Avec sélection de variable

Les calculs des procédures backward et stepwise deviennnent chronophages avec ce feature engineering...

In [ ]:
# linreg_backward_fe = lr.linreg_backward_proc(add_constant(X_train_fe), y_train, crit='BIC', verbose=True)

In [ ]:
linreg_forward_fe = lr.linreg_forward_proc(add_constant(X_train_fe), y_train, crit='BIC', verbose=True)

In [ ]:
# linreg_stepwise_fe = lr.linreg_stepwise_proc(add_constant(X_train_fe), y_train, crit='BIC', verbose=True)

In [ ]:
# print('Nombre de paramètres du modèle backward FE :', linreg_backward_fe.params.shape[0])
print('Nombre de paramètres du modèle forward FE  :', linreg_forward_fe.params.shape[0])
# print('Nombre de paramètres du modèle stepwise FE :', linreg_stepwise_fe.params.shape[0])

In [ ]:
# print('Covariables du modèle backward FE :', linreg_backward_fe.model.exog_names)
print('Covariables du modèle forward FE  :', linreg_forward_fe.model.exog_names)
# print('Covariables du modèle stepwise FE :', linreg_stepwise_fe.model.exog_names)

In [ ]:
# print(f'BIC du modèle backward FE : {linreg_backward_fe.bic:.2f}')
print(f'BIC du modèle forward FE  : {linreg_forward_fe.bic:.2f}')
# print(f'BIC du modèle stepwise FE : {linreg_stepwise_fe.bic:.2f}')

In [ ]:
# y_test_pred_linreg_backward_fe = linreg_backward_fe.predict(add_constant(X_test_fe)[linreg_backward_fe.model.exog_names])
y_test_pred_linreg_forward_fe  = linreg_forward_fe.predict(add_constant(X_test_fe)[linreg_forward_fe.model.exog_names])
# y_test_pred_linreg_stepwise_fe = linreg_stepwise_fe.predict(add_constant(X_test_fe)[linreg_stepwise_fe.model.exog_names])

In [ ]:
# RMSE_linreg_backward_fe = root_mean_squared_error(y_test, y_test_pred_linreg_backward_fe)
RMSE_linreg_forward_fe  = root_mean_squared_error(y_test, y_test_pred_linreg_forward_fe)
# RMSE_linreg_stepwise_fe = root_mean_squared_error(y_test, y_test_pred_linreg_stepwise_fe)

# print(f'RMSE régression linéaire backward FE : {RMSE_linreg_backward_fe:.2f}')
print(f'RMSE régression linéaire forward FE  : {RMSE_linreg_forward_fe:.2f}')
#print(f'RMSE régression linéaire stepwise FE : {RMSE_linreg_stepwise_fe:.2f}')

In [ ]:
# MAPE_linreg_backward_fe = mean_absolute_percentage_error(y_test, y_test_pred_linreg_backward_fe) * 100
MAPE_linreg_forward_fe  = mean_absolute_percentage_error(y_test, y_test_pred_linreg_forward_fe) * 100
# MAPE_linreg_stepwise_fe = mean_absolute_percentage_error(y_test, y_test_pred_linreg_stepwise_fe) * 100

# print(f'MAPE régression linéaire backward FE : {MAPE_linreg_backward_fe:.2f}')
print(f'MAPE régression linéaire forward FE  : {MAPE_linreg_forward_fe:.2f}')
# print(f'MAPE régression linéaire stepwise FE : {MAPE_linreg_stepwise_fe:.2f}')

### 2.5. Bilan

In [ ]:
RMSE_model = {}

RMSE_model['complète']    = RMSE_linreg_all
RMSE_model['backward']    = RMSE_linreg_backward
RMSE_model['forward']     = RMSE_linreg_forward
RMSE_model['stepwise']    = RMSE_linreg_stepwise
RMSE_model['complète FE'] = RMSE_linreg_all_fe
# RMSE_model['backward FE'] = RMSE_linreg_backward_fe
RMSE_model['forward FE']  = RMSE_linreg_forward_fe
# RMSE_model['stepwise FE'] = RMSE_linreg_stepwise_fe

RMSE_model

In [ ]:
MAPE_model = {}

MAPE_model['complète']    = MAPE_linreg_all
MAPE_model['backward']    = MAPE_linreg_backward
MAPE_model['forward']     = MAPE_linreg_forward
MAPE_model['stepwise']    = MAPE_linreg_stepwise
MAPE_model['complète FE'] = MAPE_linreg_all_fe
# MAPE_model['backward FE'] = MAPE_linreg_backward_fe
MAPE_model['forward FE']  = MAPE_linreg_forward_fe
# MAPE_model['stepwise FE'] = MAPE_linreg_stepwise_fe

MAPE_model

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(RMSE_model.keys(), RMSE_model.values())
#ax.grid()
ax.tick_params(axis='x', labelrotation=90)
ax.set_ylabel('RMSE')
plt.title('Régressions linéaires')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(RMSE_model.keys(), MAPE_model.values())
#ax.grid()
ax.tick_params(axis='x', labelrotation=90)
ax.set_ylabel('MAPE')
plt.title('Régressions linéaires')
plt.show()